In [1]:
import matplotlib
matplotlib.use('Agg')
# Remove padding around screen

from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np

from os import walk
import tensorflow as tf,numpy,keras,PIL
from PIL import Image

import sys
from os.path import dirname
sys.path.append("/home/jason/sproj/donkeycar")

import donkeycar as dk
import donkeycar.parts.datastore as datastore

import keras

import donkeycar as dk
#import parts
from donkeycar.parts.camera import PiCamera
from donkeycar.parts.transform import Lambda
from donkeycar.parts.keras import KerasCategorical, KerasRNN_LSTM, KerasRNN_Categorical, KerasHresCategorical, KerasLinear

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


using donkey v2.2.1 ...


Using TensorFlow backend.


In [80]:
a = datastore.Tub("../datasets/8track/data-4-4/data/tub_5_18-04-04/")
#b = datastore.Tub("../datasets/8track/data-4-4/data/tub_6_18-04-04/")

path_in_tub: ../datasets/8track/data-4-4/data/tub_5_18-04-04/
Tub exists: ../datasets/8track/data-4-4/data/tub_5_18-04-04/


In [113]:
a.get_num_records()

1196

In [81]:
kl = KerasLinear()
kl.load('/home/jason/sproj/d2/models/linear_8track')

# Evaluation of model
errors = []
max_range = a.get_num_records()
for i in range(1,max_range):
    true_angle = (a.get_record(i)['user/angle'])
    img_arr = a.get_record(i)['cam/image_array']
    img_arr = img_arr.reshape((1,) + img_arr.shape)
    predict_angle = kl.model.predict(img_arr)[0].item(0)
    delta = true_angle-predict_angle
    delta = delta*delta # Square the error
    errors.append(delta)
sum(errors)/max_range

0.07096916587783582

In [112]:
kl = KerasRNN_LSTM()
kl.load('/home/jason/sproj/d2/models/rnn_8track')
errors = []
count = 0
i = 1
max_range = a.get_num_records()-3
# max_range = 10
while i <= max_range-3:
    a1 = (a.get_record(i)['cam/image_array'])
    a1 = a1.reshape((1,) + a1.shape)
    
    a2 = (a.get_record(i+1)['cam/image_array'])
    a2 = a2.reshape((1,) + a2.shape)
    
    a3 = (a.get_record(i+2)['cam/image_array'])
    a3 = a3.reshape((1,) + a3.shape)
    
    x1 = np.concatenate((a1, a2, a3))
    x1 = x1.reshape((1,) + x1.shape)
    
    true_angle = (a.get_record(i)['user/angle'])
    predict_angle = kl.model.predict(x1)
    delta = true_angle-predict_angle
    delta = delta*delta # Square the error
    count+=1
    i+=1
    errors.append(delta)
sum(errors)/count

array([[0.04521056, 0.62674814]], dtype=float32)

In [122]:
tub2 = datastore.Tub("/home/jason/sproj/datasets/8track/data/tub_2_18-04-03")
tub3 = datastore.Tub("/home/jason/sproj/datasets/8track/data/tub_3_18-04-03")

tub2.get_num_records()+tub3.get_num_records()

path_in_tub: /home/jason/sproj/datasets/8track/data/tub_2_18-04-03
Tub exists: /home/jason/sproj/datasets/8track/data/tub_2_18-04-03
path_in_tub: /home/jason/sproj/datasets/8track/data/tub_3_18-04-03
Tub exists: /home/jason/sproj/datasets/8track/data/tub_3_18-04-03


23284

In [2]:
rnnkl = KerasRNN_Categorical()